#Task 1: Tokenization



In [0]:
# Download gutenberg dataset
# This is the dataset on which all your models will be trained
# https://drive.google.com/file/d/0B2Mzhc7popBga2RkcWZNcjlRTGM/edit

# Tokenization

The below task uses a regex to break a the grammer in words but the terminals do not get handled here. I have made a handler loop for breaking down sentences and add <EOS> ie End of Sentence in the end of every sentence as it woould be required in n-gram model generation.

In [476]:
import sys, re
import plotly as py
py.tools.set_credentials_file(username='sunchitsharma9',api_key='1e8V1NqDZtCw9ci0ZLil')
import plotly.graph_objs as go

def is_number(s): # Helper Functions
    try:
        float(s)
        return True
    except ValueError:
        return False

def tokenize(text):
    
    text = text.lower(); # Case Folding
    
    split_text = re.split('''a-z|
                          ,|
                          \n| |
                          \r|
                          \*|
                          --|
                          !|
                          \?|
                          ; |
                          \"|
                          _|
                          \]|
                          \[''',text)
    
    # The issue is that we have not handled "." and Dr. and such types are not handled.
    
    Dot_Handler = ["ie.","i.e.","inc.","i.o.u.","m.d.","n.b.","p.o.","u.k.","u.s.","u.s.a.","p.s.",".c","mr.","mrs.","ms.",".com","dr.",".sh",".java","st.","jr.","ph.d."]
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    
    # Web_Extentions = [".com",".in",".org",".edu",".net",".gov",".info",".biz"] Automatically Handled
    
    counter = -1;
    
    for i in split_text:
        counter+=1
        if i[-1] in punctuations:
            if i not in Dot_Handler:
                temp = re.sub(r'[^\w\s]','',i)
                split_text[counter]=temp
                split_text.insert(counter, "<EOS>")
                
    counter = -1;
    
    for i in split_text :
        if counter<len(split_text)-1:
            counter+=1
            if split_text[counter] == "<EOS>":
                t1=split_text[counter]
                split_text[counter]=split_text[counter+1]
                split_text[counter+1]=t1
                counter+=1
                
    return split_text
    

text = "i am a i am a Mr. John Johnson Jr. was born in the U.S.A but earned his Ph.D. in Israel before joining Nike Inc. as an engineer. He also worked at craigslist.org as a business analyst. Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of 0.9 it isn't don's in Israel ok."
tokens=tokenize(text)
print "-----------------------------------------------------TOKENS-----------------------------------------------------"
print tokens



-----------------------------------------------------TOKENS-----------------------------------------------------
['i', 'am', 'a', 'i', 'am', 'a', 'mr.', 'john', 'johnson', 'jr.', 'was', 'born', 'in', 'the', 'u.s.a', 'but', 'earned', 'his', 'ph.d.', 'in', 'israel', 'before', 'joining', 'nike', 'inc.', 'as', 'an', 'engineer', '<EOS>', 'he', 'also', 'worked', 'at', 'craigslist.org', 'as', 'a', 'business', 'analyst', '<EOS>', 'mr.', 'smith', 'bought', 'cheapsite.com', 'for', '1.5', 'million', 'dollars', '<EOS>', 'i.e.', 'he', 'paid', 'a', 'lot', 'for', 'it', '<EOS>', 'did', 'he', 'mind', '<EOS>', 'adam', 'jones', 'jr.', 'thinks', 'he', 'didnt', '<EOS>', 'in', 'any', 'case', '<EOS>', 'this', "isn't", 'true', '<EOS>', 'well', '<EOS>', 'with', 'a', 'probability', 'of', '0.9', 'it', "isn't", "don's", 'in', 'israel', 'ok', '<EOS>']


#Language Model and Smoothing


# Language Model, Plotting, Smoothing, Word Prediction 

In [527]:
def language_model(tokens,n,ps):
# Build your language model
# For any language model you build, print a sample of the top 10 and the bottom 10 frequently occuring N-grams.
# For any language model you build, display plots of N-Gram vs Freq sorted by frequency (descending) and log-log plots of the same
    
    gm={}
    
    for i in range(0,len(tokens)):
        temp=[]
        flag=1
        for j in range(i,n+i):
            if(tokens[j]!="<EOS>"):
                temp.append(tokens[j])
            else:
                flag=0
                break
        if(flag==0):
            i=j+1
        else:
            if tuple(temp) in gm.keys():
                gm[tuple(temp)]+=1
            else:
                gm[tuple(temp)]=1
    
    sorted_gm = dict()
    sorted_gm = sorted(gm.iteritems(),key=lambda (k,v): (v,k),reverse=True)
    
    if(ps):
        print "------Top ten N-Grams------"

        for i in range(0,9):
            print("%s\t\t:\t%d" % (sorted_gm[i][0],sorted_gm[i][1]))

        print "------Bottom ten N-Grams------"

        for i in range(len(sorted_gm)-1,len(sorted_gm)-10,-1):
            print("%s     :     %d" % (sorted_gm[i][0],sorted_gm[i][1]))
    
    return sorted_gm
    
    
##################################################################################################################
    
    
def plotme(sorted_gm):
    
    xaxis=[]
    
    yaxis=[]
    
    for i in sorted_gm:
        xaxis.append('_'.join(i[0]))
        yaxis.append(i[1])
                
    
    # Create and style traces
    trace0 = go.Scatter(
        x = xaxis,
        y = yaxis,
        name = 'N-Gram Plot',
        line = dict(
            color = ('rgb(205, 12, 24)'),
            width = 4)
    )
    
    data = [trace0]

    # Edit the layout
    layout = dict(title = 'N-Gram Frequency Plot',
                  xaxis = dict(title = 'N-Gram'),
                  yaxis = dict(title = 'Frequency'),
                  )

    fig = dict(data=data, layout=layout)
    return fig
    
    
#################################################################################################################


def complete_ngram(ngram, n):
# Given (N-1) gram, and the value 'N', print the possibilities that complete the n-gram
# and plot them in decresing order of frequency
    
    next_word=[]
    
    for i in sorted_gm:
        temp=[]
        if list(i[0])[:-1]==ngram.split():
            temp.append(i[0][-1])
            temp.append(i[1])
        if temp!=[]:
            next_word.append(temp)
    
    
    for i in range(0,n):
        if(i<len(next_word)):
            print("%s\t:\t%d" % (next_word[i][0],next_word[i][1]))
        else:
            break
            
    # NOW PLOTTING THEM
            
    xaxis=[]
    yaxis=[]
    
    for i in range(0,n):
        if(i<len(next_word)):
            xaxis.append(next_word[i][0])
            yaxis.append(next_word[i][1])
                
    
    # Create and style traces
    trace0 = go.Scatter(
        x = xaxis,
        y = yaxis,
        name = 'N-Gram Plot',
        line = dict(
            color = ('rgb(205, 12, 24)'),
            width = 4)
    )
    
    data = [trace0]

    # Edit the layout
    layout = dict(title = 'Next Word Frequency Plot',
                  xaxis = dict(title = 'Next Word'),
                  yaxis = dict(title = 'Frequency'),
                  )

    fig = dict(data=data, layout=layout)
    return fig


#####################################################################################################################


def laplace_smoothing(n_grams):
    
    numerator=0
    denominator=0
    
    sorted_gm=language_model(tokens, len(n_grams.split()),False)
    for i in sorted_gm:
        if tuple(n_grams.split())==i[0]:
            numerator=i[1]
            break
        
    for i in sorted_gm:
        if tuple(n_grams.split())[:-1]==i[0][:-1]:
            denominator+=i[1]
    
    return ((numerator+1)*1.0)/(denominator+len(tokens))
    


# only 1 of the next 3 have to be implemented

def witten_bell(n_grams):  # NOT DONE                       
  # perform Witten-Bell smoothing
    pass
  
def good_turing(n_grams):  
  # perform Good Turing smoothing
    gtt=tokenize(n_grams)
    gtt_dict={}
    count=0
    sum=0
    sorted_gm=language_model(tokens, len(gtt), False)
    for i in sorted_gm:
        if i[0] == tuple(gtt):
            count+=i[1]
        if i[1] not in gtt_dict.keys():
            gtt_dict[i[1]]=1
            sum+=1
        else:
            gtt_dict[i[1]]+=1
            sum+=1
    ans=0
    if count == 0:                           # CASE OF 0
        ans=gtt_dict[1]*1.0/sum
    else:
        nc=gtt_dict[count]                   # CASE WHEN C^n+1 exists
        if count+1 in gtt_dict.keys():
            nc1=gtt_dict[count+1]
            ans=(count+1)*1.0*nc1/nc
        else:                                # CASE WHEN C^n+1 does not exist
            MAX= gtt_dict[min(gtt_dict.keys())]
            ans=MAX*count**-2**1.0/sum
    
    return ans
    
    
####################################################################################################################    


def kneser_ney(n_grams):
  # perform Kneser-Ney smoothing
    pass


# only 1 of the next 2 have to be implemented

def backoff(n_grams):
  # perform Backoff
    one_gm = language_model(tokens,1,False)
    two_gm = language_model(tokens,2,False)
    three_gm = language_model(tokens,3,False)
    
    fd={}
    
    fd[1]=dict(one_gm)
    fd[2]=dict(two_gm)
    fd[3]=dict(three_gm)
    
    wt=tokenize(n_grams)
    
    final_backoff_score=0
    
    for i in range(0,len(wt)):
        if len(wt[i:i+3])==3:
            if tuple(wt[i:i+3]) in fd[3].keys():
                final_backoff_score+=((fd[3][tuple(wt[i:i+3])])*1.0/fd[2][tuple(wt[i:i+2])])
            elif tuple(wt[i:i+2]) in fd[2].keys():
                final_backoff_score+=((fd[2][tuple(wt[i:i+2])])*1.0/(fd[1][tuple(wt[i:i+1])]+len(fd[2].keys())))
            elif tuple(wt[i:i+1]) in fd[1].keys():
                final_backoff_score+=((fd[1][tuple(wt[i:i+1])])*1.0/(0.0003+len(fd[1].keys())))
            else:
                final_backoff_score+=0.0003
    
    return final_backoff_score
        
    
    
    pass
  
def deleted_interpolation(n_grams):
  # perform deleted Interpolation
    pass


text = "i am a i am a i am the Mr. John Johnson Jr. was born in the U.S.A but earned his Ph.D. in Israel before joining Nike Inc. as an engineer. He also worked at craigslist.org as a business analyst. Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of 0.9 it isn't don's in Israel ok."
tokens=tokenize(text) # TESTED
sorted_gm=language_model(tokens, 3, True) 



print
print "LAPLACE : "
print laplace_smoothing("i am hello")

print
print "GOOD TURING : "
print good_turing("i am a")

print
print "BACKOFF : "
print backoff("i am a i am x y x the")

print
print "NEXT POSSIBLE WORDS WITH FREQUENCIES"
fig = complete_ngram("i am",2) # FREQUENCY PLOT for next word
py.plotly.iplot(fig, filename='Next_word_plot') 

fig = plotme(sorted_gm)  # FREQUENCY PLOT 
py.plotly.iplot(fig, filename='Total Freq Plot')

 

------Top ten N-Grams------
('i', 'am', 'a')		:	2
('am', 'a', 'i')		:	2
('a', 'i', 'am')		:	2
('worked', 'at', 'craigslist.org')		:	1
('with', 'a', 'probability')		:	1
('was', 'born', 'in')		:	1
('u.s.a', 'but', 'earned')		:	1
('this', "isn't", 'true')		:	1
('thinks', 'he', 'didnt')		:	1
------Bottom ten N-Grams------
('0.9', 'it', "isn't")     :     1
('1.5', 'million', 'dollars')     :     1
('a', 'business', 'analyst')     :     1
('a', 'lot', 'for')     :     1
('a', 'probability', 'of')     :     1
('adam', 'jones', 'jr.')     :     1
('also', 'worked', 'at')     :     1
('am', 'the', 'mr.')     :     1
('as', 'a', 'business')     :     1

LAPLACE : 
0.0105263157895

GOOD TURING : 
0.2375

BACKOFF : 
2.76012355953

NEXT POSSIBLE WORDS WITH FREQUENCIES
a	:	2
the	:	1
High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~sunchitsharma9/0 or inside your plot.ly account where it is named 'Total Freq Plot'


# Task 2: Unigrams and Spelling Correction

In [537]:
def plot_ngram_freqs(ngrams):
# Calculate unigram frequencies and plot them in the descending order of frequency
# Example code for using plotly to plot frequencies
    import plotly.plotly as py
    import plotly.graph_objs as go

    # Add data
    
    unig = language_model(tokens,1,False)
    
    x_axis=[]
    y_axis=[]
    
    for i in unig:
        x_axis.append(i[0][0])
        y_axis.append(i[1])
        

    # Create and style traces
    trace0 = go.Scatter(
        x = x_axis,
        y = y_axis,
        name = 'Unigram Plot',
        line = dict(
            color = ('rgb(205, 12, 24)'),
            width = 4)
    )
    
    data = [trace0]

    # Edit the layout
    layout = dict(title = 'Unigram Plot',
                  xaxis = dict(title = 'Tokens'),
                  yaxis = dict(title = 'Frequency'),
                  )

    fig = dict(data=data, layout=layout)
    return fig

def spell_checker(word, n_grams):
# Given a word check if the spelling is correct using your Language model
# Feel free to add more helper functions for this task
    
    neighbours=[]
    alphabets = 'abcdefghijklmnopqrstuvwxyz\'';
    
    # PART A : EDIT DISTANCE
    if word in tokens:
        print "WORD IS CORRECT"
        print word # Why Correct, Correct words
    
    else:
        # ADDITION
        for i in alphabets:
            for j in range(0,len(word)+1):
                wl=list(word)
                wl.insert(j,i)
                neighbours.append("".join(wl))
        
        # DELETION
        for i in range(0,len(word)):
            wl=list(word)
            del wl[i]
            neighbours.append("".join(wl))
            
        # SWAP
        for i in range(0,len(word)-1):
            wl=list(word)
            temp=wl[i]
            wl[i]=wl[i+1]
            wl[i+1]=temp
            neighbours.append("".join(wl))
        
        # REPLACE
        for i in alphabets:
            for j in range(0,len(word)):
                wl=list(word)
                wl[j]=i
                neighbours.append("".join(wl))
                
        valid=[]        
    
        for i in neighbours:
            if i in tokens:
                valid.append(i)
        
        neighbours=[] # FREE MEMORY
        
        model=language_model(tokens,1,False)
        
        match_count={}
        sum=0
        
        for i in model:
            if i[0][0] in valid:
                match_count[i[0][0]]=i[1]
                sum+=i[1]
                
                
        print("SUGGESTION\t:\tPROBABILITY")
        print "------------------------------------"
        for i in match_count:
            print("%s \t\t:\t%.3f" % (i,match_count[i]*1.0/sum))
                
                
                
        
    # PART B : PROBABILITY OF CORRECTNESS
    
    # Generating n grams on characters
    char_tokens={}
    for i in tokens:
        if i != "<EOS>":
            for j in range(0,len(i)):
                if i[j:n_grams] not in char_tokens.keys():
                    if len(i[j:n_grams+j])==n_grams:
                        char_tokens[i[j:n_grams+j]]=1
                else:
                    if len(i[j:n_grams+j])==n_grams:
                        char_tokens[i[j:n_grams+j]]+=1

    
    # Breaking the word in grams
    corr=0
    tot=0
    for j in range(0,len(i)-n_grams):
        if len(word[j:n_grams+j])==n_grams:
            if word[j:n_grams+j] in char_tokens.keys():
                corr+=1
            tot+=1
        
    print("By N-Gram model approach the word is correct by %.02f Percent" % (corr*1.0*100/tot))

text = "i am a i am a i am the Mr. John Johnson Jr. was born in the U.S.A but earned his Ph.D. in Israel before joining Nike Inc. as an engineer. He also worked at craigslist.org as a business analyst. Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of 0.9 it isn't don's in Israel ok."
tokens=tokenize(text) # TESTED
spell_checker("hit",2)

fig = plot_ngram_freqs(1)
py.plotly.iplot(fig, filename='Unigram_plot')

SUGGESTION	:	PROBABILITY
------------------------------------
his 		:	0.333
it 		:	0.667
By N-Gram model approach the word is correct by 100.00 Percent


# Task 3 : Grammaticality Test

In [531]:
def score_grammaticality(sentence, language_model):
  # Given a sentence, Build a model from the data which can give a score of grammaticality. 
  # More grammatical the sentence, better the score
  # Feel free to add helper functions
    x=1
    c=0
    wt=tokenize(sentence)
    for i in range(0,len(wt)):
        if len(wt[i:i+3])==3:
            x= laplace_smoothing(" ".join(wt[i:i+3]))+x
            c+=1
    
    print("The Laplase gives : ")
    print x*100/c
    
    c=0
    x=1
    for i in range(0,len(wt)):
        if len(wt[i:i+3])==3:
            x= good_turing(" ".join(wt[i:i+3]))*x
            c+=1
    print
    print("The Good Turing gives : ")
    print 1*1.0/(x*c)
    
    x=backoff(sentence)
    
    print
    print("The Backoff gives : ")
    print x
    
    

text = "i am a i am a i am the Mr. John Johnson Jr. was born in the U.S.A but earned his Ph.D. in Israel before joining Nike Inc. as an engineer. He also worked at craigslist.org as a business analyst. Mr. Smith bought cheapsite.com for 1.5 million dollars, i.e. he paid a lot for it. Did he mind? Adam Jones Jr. thinks he didn't. In any case, this isn't true... Well, with a probability of 0.9 it isn't don's in Israel ok."
tokens=tokenize(text) # TESTED
sorted_gm=language_model(tokens, 3, False) 
score_grammaticality("i am a i am a i am the",sorted_gm)

The Laplase gives : 
17.3124300112

The Good Turing gives : 
7562.12551621

The Backoff gives : 
5.66666666667
